In [1]:
import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Activation
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import os
import pickle
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 64
num_classes = 10
epochs = 100

In [3]:
train=np.load("/mnt/Doc/reserch/train.npy")
test=np.load("/mnt/Doc/reserch/test.npy")
x_train = train.astype('float32')
x_test = test.astype('float32')
x_train /= 255
x_test /= 255

In [4]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (9000, 128, 128, 3)
9000 train samples
1500 test samples


In [5]:
# divide x_test into validation and test
x_val = x_test[:1200]
x_test = x_test[1200:]

In [6]:
print("validation data: {0} \ntest data: {1}".format(x_val.shape, x_test.shape))

validation data: (1200, 128, 128, 3) 
test data: (300, 128, 128, 3)


In [7]:
input_img = Input(shape=(128, 128, 3))
x = Conv2D(64, (3, 3), padding='same')(input_img)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(16, (3, 3), padding='same')(encoded)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(3, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
decoded = Activation('sigmoid')(x)

In [8]:
model = Model(input_img, decoded)
model.compile(optimizer='adam', loss='binary_crossentropy')

In [9]:
history = model.fit(x_train, x_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, x_val),
                    shuffle=True)

Train on 9000 samples, validate on 1200 samples
Epoch 1/100
9000/9000 [==============================] - 77s 9ms/step - loss: 0.5836 - val_loss: 0.5754
Epoch 2/100
9000/9000 [==============================] - 71s 8ms/step - loss: 0.5530 - val_loss: 0.5543
Epoch 3/100
9000/9000 [==============================] - 71s 8ms/step - loss: 0.5358 - val_loss: 0.5407
Epoch 4/100
9000/9000 [==============================] - 71s 8ms/step - loss: 0.5233 - val_loss: 0.5346
Epoch 5/100
9000/9000 [==============================] - 71s 8ms/step - loss: 0.5138 - val_loss: 0.5133
Epoch 6/100
9000/9000 [==============================] - 71s 8ms/step - loss: 0.5067 - val_loss: 0.5050
Epoch 7/100
9000/9000 [==============================] - 71s 8ms/step - loss: 0.5017 - val_loss: 0.4953
Epoch 8/100
9000/9000 [==============================] - 71s 8ms/step - loss: 0.4976 - val_loss: 0.4922
Epoch 9/100
9000/9000 [==============================] - 71s 8ms/step - loss: 0.4949 - val_loss: 0.4897
Epoch 10/100
900